# <span style="color:#F72585"><center>Regresión Lineal Jerárquica</center></span>


## <span style="color:#4361EE">Introducción</span>


Introducimos los modelos lineales jerárquicos y su implementación usando programación probabilística.

Tener múltiples conjuntos de mediciones relacionadas surge todo el tiempo. En psicología matemática, por ejemplo, se evalúa múltiples habilidades en la realización de  misma tarea. Po ejemplo, la resolución de un problema matemático, puede implicar habilidades de aritmética, manipulación de símbolos, comprensión de lectura, etc. 


Entonces, queremos estimar un modelo computacional / matemático que describa las habilidades del estudiante en la resolución de la tarea mediante un conjunto de parámetros.

Por lo tanto, podríamos ajustar un modelo a cada habilidad individualmente, suponiendo que no compartan similitudes; o agrupar todos los datos y calcular un modelo asumiendo que todos las habilidades son una sola, digamos habilidad matemática.

El modelamiento jerárquico permite lo mejor de ambos mundos al permitir  modelar las similitudes de los sujetos, pero también permitiendo la estimación de parámetros individuales.

En este cuaderno,  utilizaremos un ejemplo más clásico de regresión lineal jerárquica para predecir los niveles de radón en las casas.


## <span style="color:#4361EE">Los datos</span>


El conjunto de datos de radón de {cite}`gelman2003bayesian` es un clásico para el modelado jerárquico. En este conjunto de datos corresponde a la cantidad de radón, un gas radiactivo que se ha medido en diferentes hogares en todos los condados de varios estados de los Estados Unidos.

Se sabe que el gas radón es la mayor causa de cáncer de pulmón en los no fumadores. Se cree que el gas está más presente en los hogares que contienen un sótano y difiere en la cantidad presente entre los tipos de suelo. 

Aquí investigaremos estas diferencias e intentaremos hacer predicciones de los niveles de radón en diferentes condados en función del condado y la presencia de un sótano en el hogar. En este ejemplo, analizaremos Minnesota, un estado que contiene 85 condados en los que se toma un número diferente de medidas, que van de 2 a 116 mediciones por condado.

In [6]:
# importa librerías requeridas

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [7]:
# read data from a local file

data = pd.read_csv('https://raw.githubusercontent.com/AprendizajeProfundo/Estadistica-Bayesiana/master/cuadernos/datos/radon.csv')
county_names = data.county.unique()
county_idx = data.county_code.values
n_counties = len(data.county.unique())
n_counties

85

In [8]:
data.head()

,Unnamed: 0,idnum,state,state2,stfips,zip,region,typebldg,floor,room,...,pcterr,adjwt,dupflag,zipflag,cntyfips,county,fips,Uppm,county_code,log_radon
0,0,5081.0,MN,MN,27.0,55735,5.0,1.0,1.0,3.0,...,9.7,1146.499190,1.0,0.0,1.0,AITKIN,27001.0,0.502054,0,0.832909
1,1,5082.0,MN,MN,27.0,55748,5.0,1.0,0.0,4.0,...,14.5,471.366223,0.0,0.0,1.0,AITKIN,27001.0,0.502054,0,0.832909
2,2,5083.0,MN,MN,27.0,55748,5.0,1.0,0.0,4.0,...,9.6,433.316718,0.0,0.0,1.0,AITKIN,27001.0,0.502054,0,1.098612
3,3,5084.0,MN,MN,27.0,56469,5.0,1.0,0.0,4.0,...,24.3,461.623670,0.0,0.0,1.0,AITKIN,27001.0,0.502054,0,0.095310
4,4,5085.0,MN,MN,27.0,55011,3.0,1.0,0.0,4.0,...,13.8,433.316718,0.0,0.0,3.0,ANOKA,27003.0,0.428565,1,1.163151


La parte de los datos que vamos a modelar luce de la siguiente manera:

In [9]:
data[['county', 'log_radon', 'floor']].head()

,county,log_radon,floor
0,AITKIN,0.832909,1.0
1,AITKIN,0.832909,0.0
2,AITKIN,1.098612,0.0
3,AITKIN,0.095310,0.0
4,ANOKA,1.163151,0.0


Como se  puede ver, tenemos múltiples mediciones de radón (convertidas para estar en la línea real), una fila para cada casa, en un condado y si la casa tiene un sótano (floor == 0) o no (floor == 1). Nos interesa saber si tener un sótano aumenta el radón medido en la casa.

## <span style="color:#4361EE">Los Modelos</span>


### <span style="color:#4CC9F0">Agrupación de las mediciones (complete pooling)</span>


Se tratan todos los condados de la misma forma. Se estima un único nivel de radón.

Matemáticamente, ese modelo sería:

$$
y_{i} = \alpha + \beta * x_{i} + \epsilon
$$

Donde $y_i$ representa la  $i$-ésima   medida en toda Minnesota. $x_{i}$ es una variable dicotómica que indica  si la casa tiene un sótano (1) o no (0), respectivamente. Con este modelo, solo estamos estimando un intercepto y una pendiente para todas las mediciones con todos los condados agrupados. El siguiente gráfico ilustra el modelo. En el gráfico ($\theta $ representa $ (\alpha, \beta) $ en nuestro caso y $y_i$ son las mediciones en el $i$-ésimo condado).

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Estadistica-Bayesiana/master/cuadernos/imagenes/No_jerarquical_model_1.png" width="350" height="300" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo no jerárquico</p>
</figcaption>
</figure>


Para especificar este  modelo en Stan, empezamos por construir el bloque data, el cual incluye vectores de mediciones de *log-radon* ($y$) y la medida de si se tiene sótano ($x$). También incluimos el número de datos, $N=919$.


Entonces:
$$
y_i\sim N (\alpha + \beta * x_i, sigma)
$$
Y la distribución previa
$$
\begin{align*}
\alpha &\sim N(0,10)\\
\beta &\sim N(0,10)\\
\sigma &\sim N(0,10),\ \sigma>0\\
\end{align*}
$$

In [10]:
pooled_data = """
data {
  int<lower=0> N; 
  vector[N] x;
  vector[N] y;
}
"""

Ahora declaramos los parámetros. Observe que *sigma* es postivo

In [11]:
pooled_parameters = """
parameters {
  real alpha;
  real beta;
  real<lower=0> sigma;
} 
"""

Y definimos el modelo. La verosimilitud de cada observación se asume normal.

In [12]:
pooled_model = """
model {
  alpha ~ normal(0,10);
  beta ~ normal(0,10);
  sigma ~ normal(0,10);
  y ~ normal(alpha + beta * x, sigma);
}
"""

In [13]:
pooled_model_code = pooled_data + pooled_parameters + pooled_model
print(pooled_model_code)


data {
  int<lower=0> N; 
  vector[N] x;
  vector[N] y;
}

parameters {
  real alpha;
  real beta;
  real<lower=0> sigma;
} 

model {
  alpha ~ normal(0,10);
  beta ~ normal(0,10);
  sigma ~ normal(0,10);
  y ~ normal(alpha + beta * x, sigma);
}



In [14]:
log_radon = data.log_radon.values
floor_measure = data.floor.values

pooled_data_dict = {'N': len(log_radon),
               'x': floor_measure,
               'y': log_radon}

In [15]:
# import stan
import pystan

In [17]:
# compile the  model
pooled_fit = pystan.StanModel(model_code=pooled_model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7bd7f8fbc709f62f9c8c3ecea79155dd NOW.


CompileError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\BuildTools\\VC\\Tools\\MSVC\\14.29.30133\\bin\\HostX86\\x64\\cl.exe' failed with exit code 2

In [ ]:
# sample
pooled_sample = pooled_fit.sampling (data=pooled_data_dict, iter=1000, chains=4,warmup=500,thin=1)

In [ ]:
alpha_0 = pooled_sample.extract(permuted=True)['alpha'].mean(0)
beta_0 = pooled_sample.extract(permuted=True)['beta'].mean(0)

In [ ]:
#### Un Primer gráfico del modelo  estimado

In [ ]:
plt.scatter(data.floor, data.log_radon)
xvals = np.linspace(-0.2, 1.2)
plt.plot(xvals, beta_0*xvals+alpha_0, 'r--')

Note que el anterior modelo asume que todas las medidas de Minnesota tienen la misma varianza.

### <span style="color:#4CC9F0">Mediciones no agrupadas</span>


Pero, ¿qué sucede si nos interesa saber si cada  condado realmente tienen diferente comportamiento. Es decir, si cada condado es modelado de manera independiente con relaciones (pendiente) y diferentes tasas base de radón (intercepto)? 

Luego, podría decir "OK, entonces, simplemente estimaré $n$ (número de condados) diferentes regresiones, una para cada condado". Matemáticamente, ese modelo sería:

$$
radon_{i,c} = \alpha_c + \beta_c ∗ floor_{i,c} + \epsilon_c
$$

Tenga en cuenta que agregamos el subíndice $c$, por lo que estamos estimando $n$ $\alpha$'s y $\beta$'s diferentes, una para cada condado.



<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Estadistica-Bayesiana/master/cuadernos/imagenes/jerarquical_model_1.png" width="400" height="300" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo jerárquico</p>
</figcaption>
</figure>

Este es el modelo en el extremo opuesto; donde arriba supusimos que todos los condados son exactamente iguales, aquí estamos diciendo que no comparten similitudes en absoluto. Como mostramos a continuación, este tipo de modelo puede ser muy ruidoso cuando tenemos pocos datos por condado, como es el caso en este conjunto de datos.

Vamos a construir este modelo.

In [ ]:
unpooled_model_code = """
data {
  int<lower=0> N; 
  int<lower=0> N_counties;
  int<lower=1,upper=N_counties> county[N];
  vector[N] x;
  vector[N] y;
} 

parameters {
  vector[N] alpha;
  vector[N] beta;
  real<lower=0> sigma;
} 

transformed parameters {
  vector[N] y_hat;

  for (i in 1:N)
    y_hat[i] =  alpha[county[i]] + beta[county[i]] * x[i] ;
}

model {
  alpha ~ normal(0,10);
  beta  ~ normal(0,10);
  sigma ~ cauchy(0,10);
  y ~ normal(y_hat, sigma);
}"""


In [ ]:
print(unpooled_model_code)

In [ ]:
unpooled_data = {'N': len(log_radon),
                 'N_counties': n_counties,
               'county': county_idx+1, # Stan counts starting at 1
               'x': floor_measure,
               'y': log_radon}

In [ ]:
import pystan

# compile the  model
unpooled_fit = pystan.StanModel(model_code=unpooled_model_code)


In [ ]:
# sample
unpooled_sample = unpooled_fit.sampling (data=unpooled_data, iter=1000, chains=4,warmup=500,thin=1)

In [ ]:
#unpooled_estimates = pd.Series(unpooled_fit['a'].mean(0), index=mn_counties)
#unpooled_se = pd.Series(unpooled_fit['a'].std(0), index=mn_counties)

#alpha_0 = unpooled_sample.extract(permuted=True)['alpha'].mean(0)
#beta_0 = pooled_sample.extract(permuted=True)['beta'].mean(0)

In [ ]:

order = unpooled_estimates.sort_values().index

plt.scatter(range(len(unpooled_estimates)), unpooled_estimates[order])
for i, m, se in zip(range(len(unpooled_estimates)), unpooled_estimates[order], unpooled_se[order]):
    plt.plot([i,i], [m-se, m+se], 'b-')
plt.xlim(-1,86); plt.ylim(-1,4)
plt.ylabel('Radon estimate');plt.xlabel('Ordered county');

### <span style="color:#4CC9F0">Regresión Jerárquica</span>


Afortunadamente, hay un punto medio para ambos extremos. Específicamente, podemos suponer que si bien $\alpha$'s y $\beta$'s son diferentes, una para cada condado como en el caso no agrupado, todos los coeficientes comparten similitud. Podemos modelar esto asumiendo que cada coeficiente individual proviene de una distribución grupal común:

$$
\begin{align}
\alpha_c &\sim \mathcal{N}(\mu_{\alpha},\sigma^2_{\alpha})\\
\beta_c &\sim \mathcal{N}(\mu_{\beta},\sigma^2_{\beta})
\end{align}
$$


Por lo tanto, suponemos que los interceptos $\alpha$ y las pendientes $\beta$ provienen de una distribución normal centrada alrededor de su respectiva media de grupo $\mu$ con una cierta desviación estándar $\sigma^2$, cuyos valores (o más bien posteriores) de los cuales también estimamos. Es por eso que esto se llama un modelado multinivel, jerárquico o de agrupación parcial.


<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Estadistica-Bayesiana/master/cuadernos/imagenes/full_jerarquical_model_1.png" width="400" height="300" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo jerárquico completo</p>
</figcaption>
</figure>

## <span style="color:#4361EE">Modelo Jerárquico</span>


En lugar de crear modelos por separado, el modelo jerárquico crea parámetros de grupo que consideran que los condados no son completamente diferentes sino que tienen una similitud subyacente. Estas distribuciones se utilizan posteriormente para influir en la distribución de $\alpha$ y $\beta$ de cada condado.


In [ ]:
with pm.Model() as hierarchical_model:
    # Hyperpriors for group nodes
    mu_a = pm.Normal('mu_a', mu=0., sigma=100)
    sigma_a = pm.HalfNormal('sigma_a', 5.)
    mu_b = pm.Normal('mu_b', mu=0., sigma=100)
    sigma_b = pm.HalfNormal('sigma_b', 5.)

    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    a = pm.Normal('a', mu=mu_a, sigma=sigma_a, shape=n_counties)
    # Intercept for each county, distributed around group mean mu_a
    b = pm.Normal('b', mu=mu_b, sigma=sigma_b, shape=n_counties)

    # Model error
    eps = pm.HalfCauchy('eps', 5.)

    radon_est = a[county_idx] + b[county_idx]*data.floor.values

    # Data likelihood
    radon_like = pm.Normal('radon_like', mu=radon_est,
                           sigma=eps, observed=data.log_radon)

In [ ]:
varying_intercept_slope_model = """
data {
  int<lower=0> N;
  int<lower=0> J;
  vector[N] y;
  vector[N] x;
  int county[N];
}

parameters {
  real<lower=0> sigma;
  real<lower=0> sigma_a;
  real<lower=0> sigma_b;
  vector[J] a;
  vector[J] b;
  real mu_a;
  real mu_b;
}

model {
  mu_a ~ normal(0, 100);
  mu_b ~ normal(0, 100);

  a ~ normal(mu_a, sigma_a);
  b ~ normal(mu_b, sigma_b);
  y ~ normal(a[county] + b[county].*x, sigma);
}
"""

In [ ]:
unpooled_data = {'N': len(log_radon),
                 'N_counties': n_counties,
               'county': county_idx+1, # Stan counts starting at 1
               'x': floor_measure,
               'y': log_radon}

In [ ]:
intercept_slope_data = {'N': len(log_radon),
                          'J': n_counties,
                          'county': county_idx+1, # Stan counts starting at 1
                          'x': floor_measure,
                          'y': log_radon}


In [ ]:
import pystan

# compile the  model
intercept_slope_model_fit = pystan.StanModel(model_code=varying_intercept_slope_model)


In [ ]:
# sample
intercept_slope_model_sample = intercept_slope_model_fit.sampling (data=intercept_slope_data, iter=2000, chains=4,warmup=500,thin=1)

In [ ]:
xvals = np.arange(2)
b = intercept_slope_model_sample['a'].mean(axis=0)
m = intercept_slope_model_sample['b'].mean(axis=0)
for bi,mi in zip(b,m):
    plt.plot(xvals, mi*xvals + bi, 'bo-', alpha=0.4)
plt.xlim(-0.1, 1.1);

In [ ]:
intercept_slope_model_sample